# Predict a PER custom model InferenceService

This example assumes you have already deployed the sample PER custom model Inference Service.

In [1]:

%env MODEL_NAME=per-custom-model
HOSTNAME=!(kubectl get inferenceservice "per-custom-model" -o jsonpath='{.status.url}' | cut -d "/" -f 3)
%env INGRESS_HOST=localhost
%env INGRESS_PORT=8080
%env SERVICE_HOSTNAME={HOSTNAME[0]}

env: MODEL_NAME=per-custom-model
env: INGRESS_HOST=localhost
env: INGRESS_PORT=8080
env: SERVICE_HOSTNAME=per-custom-model.default.example.com


# Load and save input

In [2]:
import torch
import json
import requests
import json
import os
import numpy as np

In [3]:
array = torch.load('reduced_tronchetto_array.pt')

/tmp/ipykernel_141202/27486490.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  array = torch.load('reduced_tronchetto_array.pt')


In [4]:
# Create request message to be sent to the predictor
message_data = {}
inputs = {}
message_data["inputs"] = []
inputs["name"]="input1"
inputs["shape"] = array.shape
inputs["datatype"]="FP32" # as the given per model expects float32
inputs["data"]=array.tolist()
message_data["inputs"].append(inputs)

In [5]:
# Call predictor

service_hostname=os.environ["SERVICE_HOSTNAME"]
model_name=os.environ["MODEL_NAME"]
ingress_ip="localhost"
ingress_port=os.environ["INGRESS_PORT"]
predictor_url = f"http://{ingress_ip}:{ingress_port}/v2/models/{model_name}/infer"
request_headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Host": service_hostname,
}
response = requests.post(predictor_url, headers=request_headers, data=json.dumps(message_data))

In [6]:
response_message = json.loads(response.text)
output1 = np.array(response_message["outputs"][0]['data'], dtype=np.float32)

In [7]:
# postprocess
print(output1)

[2.2801923e+08 2.2591115e+08 2.2561736e+08 2.2002536e+08 2.1993590e+08]
